In [1]:
import os

In [2]:
import cv2

In [ ]:
from pathlib import Path

In [37]:
IMG_DIR = 'D:\\2026\\CV_Project\\smart-traffic-management-system\\IDD_Dataset\\JPEGImages'
img_path = Path(IMG_DIR) / 'frontFar' / 'BLR-2018-04-16_16-14-27_frontFar'/'0017640.jpg'

print(img_path)

D:\2026\CV_Project\smart-traffic-management-system\IDD_Dataset\JPEGImages\frontFar\BLR-2018-04-16_16-14-27_frontFar\0017640.jpg


In [38]:
image = cv2.imread(img_path)
cv2.imshow("Image",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [39]:
train_text_path ='D:\\2026\\CV_Project\\smart-traffic-management-system\\IDD_Dataset\\test.txt' 

In [ ]:
import shutil
DATA_DIR = Path(r"D:\2026\CV_Project\smart-traffic-management-system\IDD_Dataset")
IMG_DIR = DATA_DIR / "JPEGImages"
TEST_DIR = DATA_DIR / "Processed_dataset" / "test" / "images"
TEST_DIR.mkdir(parents=True, exist_ok=True)


with open(train_text_path, "r") as f:
    for idx, line in enumerate(f):
        rel_path = line.strip() + ".jpg"   

        img_path = IMG_DIR / rel_path
        dst_path = TEST_DIR / img_path.name

        print(idx, img_path)
        if not img_path.exists():
            print(" Missing:", img_path)
            continue
        shutil.copy2(img_path, TEST_DIR) 
        image = cv2.imread(str(img_path))
        if image is None:
            print("Failed to read:", img_path)
            continue

        cv2.imshow(f"image {idx}",image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        if idx ==5:
            break

0 D:\2026\CV_Project\smart-traffic-management-system\IDD_Dataset\JPEGImages\frontFar\BLR-2018-04-16_15-34-27_frontFar\0006780.jpg
1 D:\2026\CV_Project\smart-traffic-management-system\IDD_Dataset\JPEGImages\frontFar\BLR-2018-04-16_15-34-27_frontFar\0008220.jpg
2 D:\2026\CV_Project\smart-traffic-management-system\IDD_Dataset\JPEGImages\frontFar\BLR-2018-04-16_15-34-27_frontFar\0011220.jpg
3 D:\2026\CV_Project\smart-traffic-management-system\IDD_Dataset\JPEGImages\frontFar\BLR-2018-04-16_15-34-27_frontFar\000912_r.jpg
4 D:\2026\CV_Project\smart-traffic-management-system\IDD_Dataset\JPEGImages\frontFar\BLR-2018-04-16_15-34-27_frontFar\0007920.jpg
5 D:\2026\CV_Project\smart-traffic-management-system\IDD_Dataset\JPEGImages\frontFar\BLR-2018-04-16_15-34-27_frontFar\001284_r.jpg


In [ ]:
import shutil
DATA_DIR = Path(r"D:\2026\CV_Project\smart-traffic-management-system\IDD_Dataset")
LAB_DIR = DATA_DIR / "Annotations"
TEST_DIR = DATA_DIR / "Processed_dataset" / "test" / "labels"
TEST_DIR.mkdir(parents=True, exist_ok=True)

In [57]:
LABEL_ROOT = DATA_DIR / "Annotations"
FLAT_LABEL_DIR = LABEL_ROOT / "_flat_labels"

FLAT_LABEL_DIR.mkdir(exist_ok=True)

In [64]:
for idx,label_path in enumerate(LABEL_ROOT.rglob('*.xml')):
    dst: Path = FLAT_LABEL_DIR / label_path.name
    if not dst.exists():
        shutil.copy2(label_path, dst)

    if idx ==4:
        break

find the label using file_name 

In [ ]:
def find_label(label_path,img_path):
    label_name = img_path.name + '.xml'
    label_path = label_path / label_name

    if not label_path.exists():
        raise FileNotFoundError(f"Missing label for {img_path.name}")
    
    return label_path

Check duplicates

In [ ]:
from collections import Counter

counts = Counter(p.name for p in FLAT_LABEL_DIR.glob("*.png"))
dupes = [k for k,v in counts.items() if v > 1]

print("Duplicate labels:", dupes)


Check missing labels

In [ ]:
missing = []

for img in all_images:
    if img.stem not in label_index:
        missing.append(img.name)

print("Missing labels:", len(missing))


implement mapping in your XML → YOLO conversion

Define Merge Map

In [ ]:
# Merge fine-grained IDD labels into traffic-relevant classes
MERGE_MAP = {
    # vehicles
    "car": "vehicle",
    "bus": "vehicle",
    "truck": "vehicle",
    "caravan": "vehicle",
    "trailer": "vehicle",
    'autorickshaw':'vehicle',

    # two-wheelers
    "motorcycle": "two_wheeler",
    "bicycle": "two_wheeler",

    # humans
    "person": "person",
    "rider": "person",

    # infra
    "traffic_light": "traffic_light",
    "traffic_sign": "traffic_sign",

    # others
    "animal": "animal"
}



Final YOLO IDs

In [ ]:

YOLO_CLASSES = {
    "person": 0,
    "vehicle": 1,
    "two_wheeler": 2,
    "traffic_light": 3,
    "traffic_sign": 4,
    "animal": 5
}

 Use during conversion

In [ ]:
orig = obj.find("name").text.strip()
merged = MERGE_MAP.get(orig)

if merged is None:
    pass 

cls_id = YOLO_CLASSES[merged]

XML-YOLO Conversion 

In [65]:
import xml.etree.ElementTree as ET
# config
XML_DIR = FLAT_LABEL_DIR
YOLO_LABEL_PATH = DATA_DIR / "Processed_dataset" / "test" / "labels"
YOLO_LABEL_PATH.mkdir(parents=True,exist_ok=True)

#mapping
MERGE_MAP = {
    # vehicles
    "car": "vehicle",
    "bus": "vehicle",
    "truck": "vehicle",
    "caravan": "vehicle",
    "trailer": "vehicle",
    'autorickshaw':'vehicle',

    # two-wheelers
    "motorcycle": "two_wheeler",
    "bicycle": "two_wheeler",

    # humans
    "person": "person",
    "rider": "person",

    # infra
    "traffic_light": "traffic_light",
    "traffic_sign": "traffic_sign",

    # others
    "animal": "animal"
}

YOLO_CLASSES = {
    "person": 0,
    "vehicle": 1,
    "two_wheeler": 2,
    "traffic_light": 3,
    "traffic_sign": 4,
    "animal": 5
}

#Function
def convert_bbox(size,box):
    img_w,img_h = size
    xmin, ymin , xmax, ymax  = box

    x_center = ((xmin+xmax)/2)/img_w
    y_center = ((ymin+ymax)/2)/img_h

    w = (xmax-xmin)/img_w
    h = (ymax-ymin)/img_h

    return x_center,y_center,w,h

def convert_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    img_w = int(root.find("size/width").text)
    img_h = int(root.find("size/height").text)

    yolo_lines = []


    for obj in root.findall("object"):
        orig_label = obj.find("name").text.strip()

        if orig_label not in MERGE_MAP:
            continue
        merged_label = MERGE_MAP[orig_label]
        class_id = YOLO_CLASSES[merged_label]
        
        bndbox = obj.find("bndbox")
        xmin = float(bndbox.find("xmin").text)
        xmax = float(bndbox.find("xmax").text)
        ymin = float(bndbox.find("ymin").text)
        ymax = float(bndbox.find("ymax").text)

        bbox = convert_bbox(size=(img_w,img_h),box=(xmin,ymin,xmax,ymax))
        yolo_lines.append(
            f"{class_id} " + " ".join(f"{v:.6f}" for v in bbox)
        )

    return yolo_lines
# ================= MAIN LOOP =================
count = 0

for xml_file in XML_DIR.rglob("*.xml"):
    yolo_data = convert_xml(xml_file)

    if not yolo_data:
        continue  

    label_path = YOLO_LABEL_PATH / f"{xml_file.stem}.txt"
    with open(label_path, "w") as f:
        f.write("\n".join(yolo_data))

    count += 1
    if count % 1000 == 0:
        print(f"Processed {count} files")

print(f" Conversion complete: {count} label files created")

 Conversion complete: 5 label files created


In [69]:
import cv2

def draw_yolo_bboxes(img_path, label_path, class_names):
    img = cv2.imread(str(img_path))
    h, w, _ = img.shape

    with open(label_path) as f:
        for line in f:
            cls, xc, yc, bw, bh = map(float, line.split())

            # convert to pixel coords
            x_center = int(xc * w)
            y_center = int(yc * h)
            bw = int(bw * w)
            bh = int(bh * h)

            xmin = int(x_center - bw / 2)
            ymin = int(y_center - bh / 2)
            xmax = int(x_center + bw / 2)
            ymax = int(y_center + bh / 2)

            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0,255,0), 2)
            cv2.putText(
                img,
                class_names[int(cls)],
                (xmin, ymin - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0,255,0),
                2
            )

    cv2.imshow("YOLO BBox Check", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [70]:
CLASS_NAMES = list(YOLO_CLASSES.keys())

img = Path(r'D:\2026\CV_Project\smart-traffic-management-system\IDD_Dataset\JPEGImages\frontFar\BLR-2018-04-19_17-46-55_frontFar\0000420.jpg')
label = YOLO_LABEL_PATH / "0000420.txt"

draw_yolo_bboxes(img, label, CLASS_NAMES)